In [1]:
%reload_ext autoreload
%autoreload all

%aimport -torch
%aimport -matplotlib
%aimport -seaborn
%aimport -numpy
%aimport -pandas
%aimport -scipy
%aimport -lightning 

In [2]:
from eeg_snn_encoder.config import PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataset, CHBMITDataModule

# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

2025-05-07 10:18:58.065 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /root/snn-encoder-test


In [3]:
datamodule = CHBMITDataModule(dataset, batch_size=512, worker=0)

In [4]:
import os

import optuna

# Initialize the Optuna study
sampler = optuna.samplers.TPESampler(multivariate=True)
study = optuna.load_study(
    study_name="model-tuning-tbr",
    storage=os.environ["OPTUNA_CONN_STRING"],
    sampler=sampler,
)

/root/snn-encoder-test/.venv/lib/python3.12/site-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [5]:
complete_trial = study.get_trials(False, states=(optuna.trial.TrialState.COMPLETE,))
sorted_trials = sorted(complete_trial, key=lambda t: t.value)
filtered_trials = list(filter(lambda t: t.value < 16, sorted_trials))
filtered_trials

[FrozenTrial(number=22, state=1, values=[13.460037231445312], datetime_start=datetime.datetime(2025, 5, 7, 9, 50, 15, 365921), datetime_complete=datetime.datetime(2025, 5, 7, 9, 51, 29, 758253), params={'threshold': 0.043919398650706404, 'slope': 10.50493804185783, 'beta': 0.8871877886982101, 'dropout_rate1': 0.42902441176790485, 'dropout_rate2': 0.10871506022527165, 'lr': 9.182800132489358e-05, 'weight_decay': 3.0314735054464033e-06, 'scheduler_factor': 0.2975191201261136, 'scheduler_patience': 10, 'tbr_threshold': 0.957310794127068}, user_attrs={}, system_attrs={}, intermediate_values={0: 19.41507911682129, 1: 18.635786056518555, 2: 16.397323608398438, 3: 16.300153732299805, 4: 17.231199264526367, 5: 15.6973237991333, 6: 15.026949882507324, 7: 15.19406509399414, 8: 14.628440856933594, 9: 14.179835319519043, 10: 14.150229454040527, 11: 13.826264381408691, 12: 13.649250984191895, 13: 13.708549499511719, 14: 13.211064338684082, 15: 13.423517227172852, 16: 13.491031646728516, 17: 13.0993

In [ ]:
from loguru import logger

import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback

from eeg_snn_encoder.encoders import TBREncoder
from eeg_snn_encoder.models.classifier import ModelConfig
from eeg_snn_encoder.models.lightning import LitSeizureClassifier, OptimizerConfig
import torch

torch.set_float32_matmul_precision('medium')


def objective(trial: optuna.Trial) -> float:
    model_params: ModelConfig = {
        "threshold": trial.suggest_float("threshold", 0.01, 0.5),
        "slope": trial.suggest_float("slope", 1.0, 20.0),
        "beta": trial.suggest_float("beta", 0.1, 0.99),
        "dropout_rate1": trial.suggest_float("dropout_rate1", 0.1, 0.99),
        "dropout_rate2": trial.suggest_float("dropout_rate2", 0.1, 0.99),
    }

    optimizer_params: OptimizerConfig = {
        "lr": trial.suggest_float("lr", 1e-6, 1e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True),
        "scheduler_factor": trial.suggest_float("scheduler_factor", 0.1, 0.99),
        "scheduler_patience": trial.suggest_int("scheduler_patience", 1, 10),
    }

    encoder_params = {
        "threshold": trial.suggest_float("tbr_threshold", 0.1, 4),
    }

    spike_encoder = TBREncoder(**encoder_params)
    
    lit_model = LitSeizureClassifier(
        model_config=model_params,
        optimizer_config=optimizer_params,
        spike_encoder=spike_encoder,
    )

    trainer = pl.Trainer(
        max_epochs=50,
        accelerator="auto",
        devices="auto",
        strategy="auto",
        enable_model_summary=False,
        enable_checkpointing=False,
        callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_loss"), EarlyStopping(monitor="val_loss", mode="min", patience=5)],
        logger=False,
    )

    trainer.fit(lit_model, datamodule=datamodule)

    val_loss = trainer.callback_metrics["val_loss"].item()
    
    trainer.test(lit_model, datamodule=datamodule)

    test_loss = trainer.callback_metrics["test_loss"].item()
    test_acc = trainer.callback_metrics["test_acc"]
    test_f1 = trainer.callback_metrics["test_f1"]
    test_mse = trainer.callback_metrics["test_mse"]
    test_total_spikes = trainer.callback_metrics["test_total_spikes"]

    logger.info(f"Encoder: TBR Encoding,trial: {trial.number}, test_loss:{test_loss}, test_mse:{test_mse}, test_acc:{test_acc}, test_f1:{test_f1}, test_total_spikes:{test_total_spikes}")

    return val_loss

In [8]:
for trial in filtered_trials:
    study.enqueue_trial(params=trial.params)

study.optimize(objective, n_trials=len(filtered_trials))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/root/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.
/root/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[I 2025-05-07 10:26:57,665] Trial 53 pruned. Trial was pruned at epoch 7.


In [9]:
logger.info(f"Encoder: TBR Encoding,trial, best_param: {study.best_params}")
logger.info(f"Encoder: TBR Encoding,trial, best_score: {study.best_value}")

2025-05-07 10:27:10.696 | INFO     | __main__:<module>:1 - Encoder: TBR Encoding,trial, best_param: {'threshold': 0.043919398650706404, 'slope': 10.50493804185783, 'beta': 0.8871877886982101, 'dropout_rate1': 0.42902441176790485, 'dropout_rate2': 0.10871506022527165, 'lr': 9.182800132489358e-05, 'weight_decay': 3.0314735054464033e-06, 'scheduler_factor': 0.2975191201261136, 'scheduler_patience': 10, 'tbr_threshold': 0.957310794127068}
2025-05-07 10:27:11.108 | INFO     | __main__:<module>:2 - Encoder: TBR Encoding,trial, best_score: 11.944815635681152
